## Preparation

In [1]:
import pandas as pd
import re
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Load Data

In [2]:
dateparser = lambda x: pd.datetime.strptime(x, '%Y%m%d')

dat = pd.read_csv('Daten Saison 15-18 Vers. 3 - ohne Interhome.csv',sep=';',
                  dtype={'Saison':np.int32,'Buchungsnummer':np.int32,'PLZ':np.str},
                 parse_dates = ['Buchungsdatum','Anreise','Abreise'],
                 date_parser=dateparser,
                 low_memory=False)

In [3]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183511 entries, 0 to 183510
Data columns (total 31 columns):
Kundenname                  183511 non-null object
Kundenname Zeile 2          1980 non-null object
Adresse                     181558 non-null object
Adresse Zeile 2             6824 non-null object
Kundennation                183510 non-null object
PLZ                         183438 non-null object
Stadt                       181503 non-null object
Bundesland                  183511 non-null object
Telefon                     178322 non-null object
E-Mail                      167523 non-null object
Personen                    183511 non-null int64
Newsletter ja/nein          183510 non-null object
Brand                       183511 non-null object
Hausnummer                  183511 non-null int64
Produkttyp                  183511 non-null object
Haustier ja/nein            183511 non-null int64
Hausnation                  183511 non-null object
Hausgebiet                  1

## Data Cleaning

In [ ]:
area = {'Nordsee - DK':[80,81,82,83,84,85,88,89,77],
        'Ostsee und Landesinnere': [71,72,73,74,75,76,78,61,62,63],
        'Inseln': [51,52,41,42,43,44,45,46,47,48,49,31],
        'Nordsee - DE':[2],
        'Ostsee - DE':[4],
        'Sued- und Fjordnorwegen':[17,18,19,20,21,22,23],
        'Mittelnorwegen': [24,25],
        'Ostnorwegen':[11,12,13,14,15],
        'Nordnorwegen':[26,27,28],
        'Sued- und Westschweden':[90,91,93],
        'Nord- und Mittelschweden':[95,96],
         'not assigned': [30,34,94]}

country = {'Daenemark': area['Nordsee - DK']+area['Ostsee und Landesinnere'] + area['Inseln'],
           'Deutschland':area['Nordsee - DE']+area['Ostsee - DE'],
           'Norwegen': area['Sued- und Fjordnorwegen']+area['Mittelnorwegen']+
                       area['Ostnorwegen']+area['Nordnorwegen'],
           'Schweden': area['Sued- und Westschweden']+area['Nord- und Mittelschweden'],
           'not assigned': area['not assigned']}

def find_key(x, dict_):
    for ind, val in enumerate(list(dict_.values())):
        if (~np.isnan(x) and (x in val)):
            return list(dict_.keys())[ind]

#### New columns

In [4]:
# time between boking and travlling in days
dat['time_btw_book_travel_days'] = (dat['Anreise']-dat['Buchungsdatum'])/ np.timedelta64(1, 'D')

# length of travel
dat['length_travel'] = (dat['Abreise'] - dat['Anreise'])/np.timedelta64(1,'D')

# month of travelling
dat['month_of_travel'] = dat['Anreise'].apply(lambda x: x.month)



# Mietpreis in Euro
def price_handling(price):
    price = str(price)
    price_sep = price.split(',')
    return float(price_sep[0])

dat['Mietpreis'] = dat['Mietpreis'].apply(price_handling)

dat['Mietpreis_Euro'] = dat['Mietpreis'].astype(int32)
# DKR Umrechungskurs: 1 DKR = 0,134 Euro
dat.loc[dat['Währung']=='DKR','Mietpreis_Euro'] =  round(0.134*dat.loc[dat['Währung']=='DKR','Mietpreis']) 
# NKR Umrechungskurs: 1 DKR = 0,1053 Euro
dat.loc[dat['Währung']=='NKR','Mietpreis_Euro'] =  round(0.1053*dat.loc[dat['Währung']=='NKR','Mietpreis']) 
# SKR Umrechungskurs: 1 SKR = 0,09742 Euro
dat.loc[dat['Währung']=='SKR','Mietpreis_Euro'] =  round(0.09742*dat.loc[dat['Währung']=='SKR','Mietpreis']) 

# teures Haus = Personen >= 12 Leute + Pool
dat['teures_Haus'] = 'n'
dat.loc[(dat['Personen']>=12) & (dat['Produkttyp']=='1-Pool'),'teures_Haus'] = 'j'

# teure Buchung = innerhalb der 25% teuersten Buchungen
dat['teure_Buchung'] = 'n'
dat.loc[dat['Mietpreis_Euro']>=1167,'teure_Buchung'] = 'j'

In [5]:
dat['Mietpreis_Euro'].describe()

count    183511.000000
mean        931.691179
std         673.403702
min           0.000000
25%         480.000000
50%         734.000000
75%        1167.000000
max       10914.000000
Name: Mietpreis_Euro, dtype: float64

In [6]:
print(sum(dat['teures_Haus'] =='j')/dat.shape[0])
print(sum(dat['teure_Buchung'] == 'j')/dat.shape[0])

0.02484319741050945
0.2502465792241337


In [7]:
def date_handling(date):
    return str(date.year) + '/' + str(date.month).zfill(2)

dat['Anreise_month'] = dat['Anreise'].apply(date_handling)

#### Phone number handling

In [8]:
def phone_handling(phone):
    phone = str(phone)
    regex = re.compile('[^a-zA-Z0-9]')
    phone = regex.sub('',phone)
    
    if len(phone) < 5:
        return '0000'
    
    
    if (phone[0] == '4') & (phone[1] == '9'):
        phone = phone[2:]
    
    if phone[0] == '0':
        phone = phone[1:]
    
    return phone

dat['Telefon'] = dat['Telefon'].apply(phone_handling)

#### E-mail

In [9]:
def email_handling(email):
    email = str(email)
    email = email.lower()
    email = email.strip()
    
    if ('@' not in email) | ('.' not in email) :
        email = '0000'
    return email

dat['E-Mail '] = dat['E-Mail '].apply(email_handling)

#### Kundenname

In [10]:
def name_handling_firstname(name):
    name = str(name)
    name =  name.lower()
    
    komma1_split = name.split(', ')
    komma2_split = name.split(',')
    slash_split = name.split('/')
    doublespace_split = name.split('  ')
    space_split = name.split(' ')
    
    if len(komma1_split)>1:
        return komma1_split[-1]
    if len(komma2_split)>1:
        return komma2_split[-1]
    if len(slash_split)>1:
        return slash_split[-1]
    if len(doublespace_split)>1:
        return doublespace_split[-1]
        
    return space_split[0]

def name_handling_lastname(name):
    name = str(name)
    name =  name.lower()
    
    komma1_split = name.split(', ')
    komma2_split = name.split(',')
    slash_split = name.split('/')
    doublespace_split = name.split('  ')
    space_split = name.split(' ')
    
    if len(komma1_split)>1:
        return komma1_split[0]
    if len(komma2_split)>1:
        return komma2_split[0]
    if len(slash_split)>1:
        return slash_split[0]
    if len(doublespace_split)>1:
        return doublespace_split[0]
        
    return space_split[-1]



dat['Name1'] = dat['Kundenname'].apply(name_handling_firstname)
dat['Name2'] = dat['Kundenname'].apply(name_handling_lastname)
dat['Kundenname'] = dat['Kundenname'].str.lower()
dat['Kundenname Zeile 2 '] = dat['Kundenname Zeile 2 '].str.lower()

#### Adresse

In [11]:
#print(dat.groupby('Adresse Zeile 2')['Brand'].count())
print('Adresse ist evtl. nicht notwendig.')
print('In \'Adresszeile 2 \' steht viel Mist.')

Adresse ist evtl. nicht notwendig.
In 'Adresszeile 2 ' steht viel Mist.


#### PLZ handling

PLZ cleaning ist nur für deutsche PLZ gültig. Deshalb wird der Datensatz gesplittet in 'de' und 'notde'.

In [12]:
dat_de = dat.loc[(dat['Kundennation'] == 'DE') | (dat['Kundennation'] == 'D')].copy()
dat_notde = dat.loc[-((dat['Kundennation'] == 'DE') | (dat['Kundennation'] == 'D'))].copy()

##### Non-German Customers

In [13]:
print('Anzahl der Buchungen aus dem Ausland:')
print('Es gibt',dat_notde.shape[0], 'Buchungen aus dem Ausland, sortiert nach Nationen:')
print(dat_notde.groupby(['Kundennation'])['Abreise'].count())
print()
print('Häufigste Buchungen nach PLZ:')
print(dat_notde.groupby('PLZ')['Abreise'].count().sort_values(ascending=False)[:5])

Anzahl der Buchungen aus dem Ausland:
Es gibt 115 Buchungen aus dem Ausland, sortiert nach Nationen:
Kundennation
AT     7
BE    11
CH    12
CN     1
CS     1
DK    45
FR     2
GB     1
LU     3
NL    24
PL     4
SE     2
US     1
Name: Abreise, dtype: int64

Häufigste Buchungen nach PLZ:
PLZ
4600     28
8000      5
0000      4
00000     2
3765      1
Name: Abreise, dtype: int64


Split data set into German and non_German customers.

Es gibt 28 Buchungen von Spitzke Scandinavia A/S mit Unternehmenssitz in Dänemark, die zu einem deutschen Bauunternehmen für Bahninfrastruktur gehört.

Es gibt 5 Buchungen der STRABAG AG, ein börsennotiertes österreichisches Unternehmen mit Hauptsitz in Wien. Strabag ist eines der größten Bauunternehmen Europas.

In [14]:
#df = df.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None)
def PLZ_handling_de(PLZ):
    PLZ = str(PLZ)
    
    noPLZ = ['12345','11111','22222','44444','55555','66666','77777','88888','99999']
    
    if (len(PLZ) < 5) | (PLZ in noPLZ):
        return '00000'
    elif len(PLZ.split('-'))>1:
        return PLZ.split('-')[1]
    else:
        return PLZ

dat_de['PLZ'] = dat_de['PLZ'].apply(PLZ_handling_de)

## Saving Data

In [15]:
import pickle
dat.to_pickle('allDat.pkl')
dat_de.to_pickle('deDat.pkl')